In [ ]:
# Ex. 6-1 四連通範例
n = 1  
# a = [
#     [0, 0, 0, 0, 0, 0],
#     [0, 0, 1, 0, 0, 1],
#     [1, 1, 1, 0, 1, 1],
#     [0, 0, 1, 0, 0, 1],
#     [0, 1, 1, 0, 1, 1]
# ]

a = [
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1],
    [1, 1, 1, 0, 1, 1],
    [0, 0, 1, 0, 0, 1],
    [0, 1, 1, 0, 1, 1]
]

# 在每列前面插入 0（方便邊界判斷）
for q in a:
    q.insert(0, 0)

# 生成標籤矩陣 b
rows = len(a)
cols = len(a[0])
b = [[0] * cols for _ in range(rows)]

# 記錄等價集合
e = []

# 第 1 階段：由上至下、由左至右掃描
for i in range(1, rows):
    for j in range(1, cols):
        if a[i][j] == 0:
            continue

        top = b[i-1][j]
        left = b[i][j-1]

        if top == 0 and left == 0:
            # 上、左都沒標籤 → 新標籤
            b[i][j] = n
            n += 1
        elif top != 0 and left == 0:
            # 上有、左無 → 用上面的標籤
            b[i][j] = top
        elif top == 0 and left != 0:
            # 上無、左有 → 用左邊的標籤
            b[i][j] = left
        else:
            # 上、左都有標籤
            if top == left:
                b[i][j] = top
            else:
                # 標籤不同
                b[i][j] = top
                e.append((top, left))  # 紀錄等價集合
# 將 e 的 pair 轉換成群組，例如 (1,3),(1,5) → [1,3,5]
groups = []
for a_, b_ in e:
    merged = False
    for g in groups:
        if a_ in g or b_ in g:
            g.update([a_, b_])
            merged = True
            break
    if not merged:
        groups.append(set([a_, b_]))

# 檢查是否還有可以合併的集合（防止多層等價）
changed = True
while changed:
    changed = False
    for i in range(len(groups)):
        for j in range(i+1, len(groups)):
            if groups[i] & groups[j]:  # 有交集
                groups[i].update(groups[j])
                del groups[j]
                changed = True
                break
        if changed:
            break

# 轉成 list of lists
groups = [sorted(list(g)) for g in groups]

# 輸出結果
print("原始矩陣 a：")
for row in a:
    print(row)

print("\n第一階段標記矩陣 b：")
for row in b:
    print(row)

print("\n合併後的等價集合群組：", groups)

label_map = {}
for g in groups:
    rep = g[0]
    for x in g[1:]:
        label_map[x] = rep

# ===== 套用等價關係到 b =====
for i in range(rows):
    for j in range(cols):
        if b[i][j] in label_map:
            b[i][j] = label_map[b[i][j]]

print("\n第二階段合併後矩陣 b：")
for row in b:
    print(row)


In [ ]:
# Ex. 6-2 八連通範例

a = [
    [0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 1],
    [1, 1, 1, 0, 1, 1],
    [0, 0, 1, 0, 0, 1],
    [0, 1, 1, 0, 1, 1]
]
# a = [
#     [0, 0, 0, 0, 0, 0],
#     [0, 0, 0, 0, 0, 1],
#     [1, 1, 1, 0, 1, 1],
#     [0, 0, 1, 0, 0, 1],
#     [0, 1, 1, 0, 1, 1]
# ]

# pad left column of zeros to simplify neighbor indexing (so indices start at 1..)
for row in a:
    row.insert(0, 0)

rows = len(a)
cols = len(a[0])

# initialize label matrix for first pass
b1 = [[0]*cols for _ in range(rows)]

# union-find helper for equivalence resolution
parent = {}

def make_set(x):
    if x not in parent:
        parent[x] = x

def find(x):
    # path compression
    while parent[x] != x:
        parent[x] = parent[parent[x]]
        x = parent[x]
    return x

def union(x, y):
    rx = find(x)
    ry = find(y)
    if rx == ry:
        return
    # attach larger to smaller to keep representative small
    if rx < ry:
        parent[ry] = rx
    else:
        parent[rx] = ry

next_label = 1
equivalence_pairs = []

# First pass: consider 8-connected neighbors that are already processed:
# neighbors: top-left (i-1,j-1), top (i-1,j), top-right (i-1,j+1), left (i,j-1)
for i in range(1, rows):
    for j in range(1, cols):
        if a[i][j] == 0:
            continue
        neighbors = []
        # check bounds for top-right
        # top-left
        if b1[i-1][j-1] != 0:
            neighbors.append(b1[i-1][j-1])
        # top
        if b1[i-1][j] != 0:
            neighbors.append(b1[i-1][j])
        # top-right (ensure j+1 < cols)
        if j+1 < cols and b1[i-1][j+1] != 0:
            neighbors.append(b1[i-1][j+1])
        # left
        if b1[i][j-1] != 0:
            neighbors.append(b1[i][j-1])
        
        neighbors = sorted(set(neighbors))
        if not neighbors:
            # new label
            b1[i][j] = next_label
            make_set(next_label)
            next_label += 1
        else:
            # assign smallest neighbor label
            assigned = neighbors[0]
            b1[i][j] = assigned
            # if multiple distinct neighbor labels, record equivalences
            if len(neighbors) > 1:
                for nb in neighbors[1:]:
                    # union in uf and record pair
                    make_set(nb)
                    make_set(assigned)
                    equivalence_pairs.append((assigned, nb))
                    union(assigned, nb)

# collect equivalence groups from union-find
groups = {}
for lbl in sorted(parent.keys()):
    root = find(lbl)
    groups.setdefault(root, []).append(lbl)
groups_list = [sorted(v) for k,v in groups.items()]

# build mapping from old label to representative (choose smallest member of its group)
label_map = {}
for group in groups_list:
    rep = min(group)
    for x in group:
        label_map[x] = rep

# second pass: relabel b1 into b2 using label_map (also compress labels to sequential integers)
b2 = [[0]*cols for _ in range(rows)]
for i in range(rows):
    for j in range(cols):
        lbl = b1[i][j]
        if lbl == 0:
            b2[i][j] = 0
        else:
            if lbl in label_map:
                b2[i][j] = label_map[lbl]
            else:
                b2[i][j] = lbl

# optional: compress labels to sequential 1..k
unique_labels = sorted({b2[i][j] for i in range(rows) for j in range(cols) if b2[i][j]!=0})
remap = {old:new+1 for new,old in enumerate(unique_labels)}
b2_compact = [[(remap[lbl] if lbl in remap else 0) for lbl in row] for row in b2]

# print results
print("Padded input matrix a (with left padding):")
for row in a:
    print(row)
print("\nFirst-pass labels (b1):")
for row in b1:
    print(row)
print("\nEquivalence groups (after union-find):")
print(groups_list)
print("\nSecond-pass labels (representative labels applied):")
for row in b2:
    print(row)
# print("\nSecond-pass labels (compacted sequential labels):")
# for row in b2_compact:
#     print(row)
